In [ ]:
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import BayesianRidge
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# importing datasets

In [ ]:
activity = pd.read_csv('/content/drive/MyDrive/SUMMER_RESEARCH/Data/FitbitActivity(1-30-20).csv', parse_dates=['datadate'])
sleep = pd.read_csv('/content/drive/MyDrive/SUMMER_RESEARCH/Data/FitbitSleep(1-30-20).csv', parse_dates=['dataDate'])
gt = pd.read_csv('/content/drive/MyDrive/SUMMER_RESEARCH/Data/BasicSurvey(3-6-20).csv', low_memory=False)

In [ ]:
sleep = sleep.rename(columns={'dataDate': 'datadate'})

activity_sleep_merged = pd.merge(activity, sleep, on=['egoid', 'datadate'], how='inner')

start_dates = gt.filter(regex=r'StartDate')

# manually parsing the start dates

In [ ]:
def value_to_date(x):
  def month(month_abbr):
    month_map = {
      'jan': '01',
      'feb': '02',
      'mar': '03',
      'apr': '04',
      'may': '05',
      'jun': '06',
      'jul': '07',
      'aug': '08',
      'sep': '09',
      'oct': '10',
      'nov': '11',
      'dec': '12'
    }
    return month_map.get(month_abbr.lower(), 'Invalid month')

  return datetime.datetime.strptime(x[:2] + '-' + month(x[2:5]) + '-' + x[5:9], '%d-%m-%Y').date()

In [ ]:
start_dates = start_dates.map(value_to_date, na_action='ignore')

In [ ]:
activity_sleep_merged.insert(1, 'date_difference', 0)
activity_sleep_merged.insert(2, 'survey', np.nan)
activity_sleep_merged['target_depression'] = np.nan
activity_sleep_merged['target_anxiety'] = np.nan
activity_sleep_merged['target_sleep'] = np.nan

In [ ]:
start_dates.drop('StartDate_7', axis=1, inplace=True)
start_dates.insert(0, 'egoid', gt['egoid'])

In [ ]:
cesd_groups = gt.filter(regex=r'CESDGroup')
cesd_groups = cesd_groups.map(lambda x: 0 if x=='Not depressed' else 1, na_action='ignore')

stai_groups = gt.filter(regex=r'STAITraitGroup')
stai_groups = stai_groups.map(lambda x: 0 if x=='not anxious' else 1, na_action='ignore')

psqi_groups = gt.filter(regex=r'PSQIGroup')
psqi_groups = psqi_groups.map(lambda x: 0 if x=='Good Sleep' else 1, na_action='ignore')

In [ ]:
target_dates = pd.concat([start_dates, cesd_groups, stai_groups, psqi_groups], axis=1)

# labeling the data algorithm (survey wave, depression, anxiety, and sleep)

In [ ]:
def label_data(features_df, target_df):

    merged_df = pd.merge(features_df, target_df, on='egoid', how='left')

    conditions_1 = [
        (merged_df['datadate'] <= merged_df['StartDate_1']),
        (merged_df['datadate'] <= merged_df['StartDate_2']),
        (merged_df['datadate'] <= merged_df['StartDate_4']),
        (merged_df['datadate'] <= merged_df['StartDate_6']),
        (merged_df['datadate'] <= merged_df['StartDate_8']),
    ]

    conditions_2 = [
        (merged_df['datadate'] <= merged_df['StartDate_1']),
        (merged_df['datadate'] <= merged_df['StartDate_2']),
        (merged_df['datadate'] <= merged_df['StartDate_3']),
        (merged_df['datadate'] <= merged_df['StartDate_4']),
        (merged_df['datadate'] <= merged_df['StartDate_5']),
    ]

    choices_depression = [
        merged_df['CESDGroup_1'],
        merged_df['CESDGroup_2'],
        merged_df['CESDGroup_4'],
        merged_df['CESDGroup_6'],
        merged_df['CESDGroup_8'],
    ]

    choices_anxiety = [
        merged_df['STAITraitGroup_1'],
        merged_df['STAITraitGroup_2'],
        merged_df['STAITraitGroup_4'],
        merged_df['STAITraitGroup_6'],
        merged_df['STAITraitGroup_8'],
    ]

    choices_sleep = [
        merged_df['PSQIGroup_1'],
        merged_df['PSQIGroup_2'],
        merged_df['PSQIGroup_3'],
        merged_df['PSQIGroup_4'],
        merged_df['PSQIGroup_5'],
    ]

    choices_surveys = [1, 2, 4, 6, 8]

    features_df.loc[:, 'target_depression'] = np.select(conditions_1, choices_depression, default=np.nan)
    features_df.loc[:, 'target_anxiety'] = np.select(conditions_1, choices_anxiety, default=np.nan)
    features_df.loc[:, 'target_sleep'] = np.select(conditions_2, choices_sleep, default=np.nan)
    features_df.loc[:, 'survey'] = np.select(conditions_1, choices_surveys, default=np.nan)


    startdate_cols = ['StartDate_1', 'StartDate_2', 'StartDate_4', 'StartDate_6', 'StartDate_8']
    startdate_df = merged_df[startdate_cols]

    condition_matrix = pd.DataFrame({col: (merged_df['datadate'] <= merged_df[col]) for col in startdate_cols})


    def first_matching_date(row):
        for col in startdate_cols:
            if row[col]:
                return merged_df.loc[row.name, col]
        return np.nan

    first_dates = condition_matrix.apply(first_matching_date, axis=1)

    merged_df['datadate'] = pd.to_datetime(merged_df['datadate'], errors='coerce')
    first_dates = pd.to_datetime(first_dates, errors='coerce')

    date_difference = (first_dates - merged_df['datadate']).dt.days

    features_df.loc[:, 'date_difference'] = date_difference


    return features_df


In [ ]:
label_data(activity_sleep_merged, target_dates)

/tmp/ipython-input-586077297.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 8.  7.  7. ... nan nan nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  features_df.loc[:, 'date_difference'] = date_difference


egoid  date_difference  survey   datadate  complypercent   meanrate  \
0       44869              8.0     1.0 2015-07-28             91  86.669746   
1       44869              7.0     1.0 2015-07-29             97  87.975342   
2       44869              7.0     1.0 2015-07-29             97  87.975342   
3       44869              6.0     1.0 2015-07-30            100  83.281448   
4       44869              5.0     1.0 2015-07-31             80  91.707787   
...       ...              ...     ...        ...            ...        ...   
333284  35925              0.0     6.0 2017-11-13             85  70.777321   
333285  35925              NaN     NaN 2017-11-16             82  71.740677   
333286  35925              NaN     NaN 2017-11-26             70  73.942635   
333287  35925              NaN     NaN 2017-12-17             64  73.434593   
333288  35925              NaN     NaN 2017-12-21             59  82.506462   

           sdrate    steps  floors  sedentaryminutes  ...  timeoutofbed  \
0       11.979352   4034.0     1.0            1034.0  ...      10:27:00   
1        8.990446   3702.0     0.0             769.0  ...      07:21:30   
2        8.990446   3702.0     0.0             769.0  ...      16:44:30   
3       16.863153   3660.0     0.0             795.0  ...      14:31:30   
4       12.464956   6016.0    13.0             954.0  ...      06:27:30   
...           ...      ...     ...               ...  ...           ...   
333284  15.921599   4748.0     5.0             658.0  ...      09:15:00   
333285  16.367147   6612.0    13.0             694.0  ...      08:20:30   
333286  20.251505   9676.0    14.0             920.0  ...      08:43:00   
333287  15.791707   4745.0     7.0            1193.0  ...      19:11:30   
333288  17.774559  13553.0    33.0             977.0  ...      16:10:00   

        bedtimedur  minstofallasleep  minsafterwakeup  minsasleep  minsawake  \
0              243                 0                0         232         11   
1              236                 1                4         216         15   
2              282                 3                0         267         12   
3              485                 4                0         451         30   
4              108                 0                0         100          8   
...            ...               ...              ...         ...        ...   
333284         571                 0                0         533         38   
333285         545                 0                1         489         55   
333286         265                 6                1         249          9   
333287          77                 2                3          70          2   
333288         138                 0                1         115         22   

        Efficiency  target_depression  target_anxiety  target_sleep  
0         0.954733                0.0             0.0           0.0  
1         0.935065                0.0             0.0           0.0  
2         0.956989                0.0             0.0           0.0  
3         0.937630                0.0             0.0           0.0  
4         0.925926                0.0             0.0           0.0  
...            ...                ...             ...           ...  
333284    0.933450                NaN             NaN           NaN  
333285    0.898897                NaN             NaN           NaN  
333286    0.965116                NaN             NaN           NaN  
333287    0.972222                NaN             NaN           NaN  
333288    0.839416                NaN             NaN           NaN  

[333289 rows x 32 columns]

# manually parsing timetobed and timeoutofbed

In [ ]:
# checking
print('timetobed example:', activity_sleep_merged.loc[0, 'timetobed'])
print('type:', type(activity_sleep_merged.loc[0, 'timetobed']))

timetobed example: 06:25:00
type: <class 'str'>


In [ ]:
activity_sleep_merged['timetobed'] = pd.to_datetime(activity_sleep_merged['timetobed'], format='%H:%M:%S').dt.time
activity_sleep_merged['timeoutofbed'] = pd.to_datetime(activity_sleep_merged['timeoutofbed'], format='%H:%M:%S').dt.time

In [ ]:
# checking
print('timetobed example:', activity_sleep_merged.loc[0, 'timetobed'])
print('type:', type(activity_sleep_merged.loc[0, 'timetobed']))

timetobed example: 06:25:00
type: <class 'datetime.time'>


In [ ]:
convert_datetime_to_float = lambda x: x.hour + x.minute / 60 if pd.notnull(x) else np.nan

activity_sleep_merged['timetobed'] = (activity_sleep_merged['timetobed'].apply(convert_datetime_to_float))
activity_sleep_merged['timeoutofbed'] = (activity_sleep_merged['timeoutofbed'].apply(convert_datetime_to_float))

activity_sleep_merged['timetobed'] = activity_sleep_merged['timetobed'].round(2)
activity_sleep_merged['timeoutofbed'] = activity_sleep_merged['timeoutofbed'].round(2)

In [ ]:
# checking
print('timetobed example:', activity_sleep_merged.loc[0, 'timetobed'])
print('type:', type(activity_sleep_merged.loc[0, 'timetobed']))

timetobed example: 6.42
type: <class 'numpy.float64'>


# dropping null targets

In [ ]:
activity_sleep_merged.dropna(how='any', subset=['target_depression', 'target_anxiety', 'target_sleep'], inplace=True)

In [ ]:
datadate = activity_sleep_merged[['datadate']]
activity_sleep_merged.drop(columns='datadate', inplace=True)

# imputing using Baysian Ridge

In [ ]:
imputer = IterativeImputer(estimator=BayesianRidge(), random_state=23)
imputed_array = imputer.fit_transform(activity_sleep_merged)
df = pd.DataFrame(imputed_array, columns=activity_sleep_merged.columns)

In [ ]:
df.insert(2, 'datadate' ,datadate)

# performing mean aggregation based on egoid and survey wave

In [ ]:
def compute_weights(date_difference, method='inverse'):
    if method == 'inverse':
        return 1 / (date_difference + 1)  # Avoid div by zero
    elif method == 'exp':
        return np.exp(-0.1 * date_difference)  # λ=0.1 decay rate
    else:
        raise ValueError("Unknown weighting method")

def weighted_mean(x, w):
    return np.sum(w * x) / np.sum(w)

def weighted_std(x, w):
    mean = weighted_mean(x, w)
    return np.sqrt(np.sum(w * (x - mean) ** 2) / np.sum(w))

def weighted_var(x, w):
    mean = weighted_mean(x, w)
    return np.sum(w * (x - mean) ** 2) / np.sum(w)

def weighted_median(x, w):
    sorted_idx = np.argsort(x)
    x_sorted, w_sorted = x[sorted_idx], w[sorted_idx]
    cum_w = np.cumsum(w_sorted)
    cutoff = np.sum(w) / 2.0
    return x_sorted[np.searchsorted(cum_w, cutoff)]

# ---- Main aggregation ----
df['weight'] = compute_weights(df['date_difference'], method='inverse')

# Columns to exclude from statistics
exclude_cols = [
    'egoid', 'survey', 'weight',
    'target_depression', 'target_anxiety', 'target_sleep'
]

# Identify numeric Fitbit features (exclude egoid + others)
feature_cols = df.select_dtypes(include=[np.number]).columns.difference(exclude_cols)

# Group keys (egoid and survey only used for grouping)
group_cols = ['egoid', 'survey', 'target_depression', 'target_anxiety', 'target_sleep']

# Aggregation dictionary
agg_funcs = {}
for col in feature_cols:
    agg_funcs[f'{col}_wmean']   = (col, lambda x, c=col: weighted_mean(df.loc[x.index, c], df.loc[x.index, 'weight']))
    agg_funcs[f'{col}_wstd']    = (col, lambda x, c=col: weighted_std(df.loc[x.index, c], df.loc[x.index, 'weight']))
    agg_funcs[f'{col}_wvar']    = (col, lambda x, c=col: weighted_var(df.loc[x.index, c], df.loc[x.index, 'weight']))
    agg_funcs[f'{col}_wmedian'] = (col, lambda x, c=col: weighted_median(df.loc[x.index, c].values, df.loc[x.index, 'weight'].values))
    agg_funcs[f'{col}_min']     = (col, 'min')
    agg_funcs[f'{col}_max']     = (col, 'max')

# Perform vectorized groupby aggregation
weighted_agg_df = df.groupby(group_cols).agg(**agg_funcs).reset_index()


In [ ]:
weighted_agg_df

egoid  survey  target_depression  target_anxiety  target_sleep  \
0     10237.0     1.0                1.0             1.0           1.0   
1     10237.0     2.0                1.0             1.0           1.0   
2     10237.0     6.0                1.0             1.0           1.0   
3     10469.0     4.0                1.0             1.0           1.0   
4     10469.0     6.0                1.0             1.0           1.0   
...       ...     ...                ...             ...           ...   
1256  99733.0     4.0                0.0             0.0           0.0   
1257  99733.0     6.0                0.0             0.0           0.0   
1258  99978.0     1.0                0.0             0.0           1.0   
1259  99978.0     2.0                0.0             0.0           0.0   
1260  99978.0     4.0                0.0             0.0           0.0   

      Efficiency_wmean  Efficiency_wstd  Efficiency_wvar  Efficiency_wmedian  \
0             0.902246         0.005385         0.000029            0.896861   
1             0.952991         0.036473         0.001330            0.966019   
2             0.961212         0.035438         0.001256            0.966732   
3             0.954070         0.042036         0.001767            0.971015   
4             0.930112         0.038706         0.001498            0.937086   
...                ...              ...              ...                 ...   
1256          0.944172         0.023590         0.000557            0.942982   
1257          0.959940         0.020459         0.000419            0.961637   
1258          0.947091         0.038223         0.001461            0.945170   
1259          0.927323         0.027427         0.000752            0.934066   
1260          0.928728         0.032694         0.001069            0.930754   

      Efficiency_min  ...  timetobed_wvar  timetobed_wmedian  timetobed_min  \
0           0.896861  ...       98.010000               4.10           4.10   
1           0.861538  ...      104.101208              10.92           0.02   
2           0.690299  ...      102.461830               9.82           0.00   
3           0.661417  ...       43.607813               5.17           1.00   
4           0.824701  ...       39.607659               5.02           2.30   
...              ...  ...             ...                ...            ...   
1256        0.742857  ...       78.696778               0.98           0.00   
1257        0.912281  ...       77.111222               1.18           0.07   
1258        0.890736  ...       63.989284               0.47           0.00   
1259        0.837363  ...       28.511567               1.90           0.03   
1260        0.827586  ...       87.060038               2.37           0.00   

      timetobed_max  veryactiveminutes_wmean  veryactiveminutes_wstd  \
0             23.90                57.000000                0.000000   
1             23.98                37.817636               22.186755   
2             23.98                52.141663               36.861908   
3             23.62                 5.783356                6.816406   
4             22.72                 9.642634               11.567045   
...             ...                      ...                     ...   
1256          23.98                22.790929               15.382031   
1257          23.97                15.172623               14.762491   
1258          23.57                21.816444               24.995387   
1259          23.95                24.001621               27.679903   
1260          23.98                30.498452               24.518872   

      veryactiveminutes_wvar  veryactiveminutes_wmedian  \
0                   0.000000                       57.0   
1                 492.252105                       31.0   
2                1358.800288                       48.0   
3                  46.463387                        3.0   
4                 133.796524                     

In [ ]:
weighted_agg_df.drop(columns=['egoid', 'survey'], inplace=True)

weighted_agg_df.to_csv('all_stats.csv', index=False)
!cp all_stats.csv "/content/drive/MyDrive/SUMMER_RESEARCH/Data"

In [ ]:
target_cols = ['target_depression', 'target_anxiety', 'target_sleep']

# Function to filter and drop egoid & survey
def filter_stats(df, stat_suffix):
    cols = target_cols + [col for col in df.columns if col.endswith(stat_suffix)]
    return df[cols]

df_wmean   = filter_stats(weighted_agg_df, '_wmean')
df_wstd    = filter_stats(weighted_agg_df, '_wstd')
df_wvar    = filter_stats(weighted_agg_df, '_wvar')
df_wmedian = filter_stats(weighted_agg_df, '_wmedian')
df_min     = filter_stats(weighted_agg_df, '_min')
df_max     = filter_stats(weighted_agg_df, '_max')

# Saving dataframes as csv

In [ ]:
import os

# Directory to save CSVs
save_dir = "/content/drive/MyDrive/SUMMER_RESEARCH/Data"
os.makedirs(save_dir, exist_ok=True)

# Dictionary of DataFrames and their file name tags
dfs = {
    'wmean': df_wmean,
    'wstd': df_wstd,
    'wvar': df_wvar,
    'wmedian': df_wmedian,
    'min': df_min,
    'max': df_max
}

# Save each DataFrame as CSV
for name, df in dfs.items():
    file_path = os.path.join(save_dir, f"{name}.csv")
    df.to_csv(file_path, index=False)
    print(f"Saved: {file_path}")


Saved: /content/drive/MyDrive/SUMMER_RESEARCH/Data/wmean.csv
Saved: /content/drive/MyDrive/SUMMER_RESEARCH/Data/wstd.csv
Saved: /content/drive/MyDrive/SUMMER_RESEARCH/Data/wvar.csv
Saved: /content/drive/MyDrive/SUMMER_RESEARCH/Data/wmedian.csv
Saved: /content/drive/MyDrive/SUMMER_RESEARCH/Data/min.csv
Saved: /content/drive/MyDrive/SUMMER_RESEARCH/Data/max.csv
